In [8]:
import osmnx as ox
import pandas as pd
import folium
import branca.colormap as cm

# 获取巴尔的摩的交通路线图
G = ox.graph.graph_from_place("Baltimore, Maryland, USA", network_type="drive")


In [9]:

# 加载公交站数据
bus_stops = pd.read_csv("2025_Problem_D_Data\\Bus_Stops.csv")

# 创建Folium地图
gdf_nodes = ox.graph_to_gdfs(G, nodes=True, edges=False)
center_lat = gdf_nodes['y'].mean()
center_lon = gdf_nodes['x'].mean()
mm = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# # 创建第一个图层（Layer 1）
# layer1 = folium.FeatureGroup(name='Layer 1 - Points')

# # 添加道路网络到地图
# edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
# folium.GeoJson(
#     edges[['geometry']],
#     style_function=lambda x: {
#         'color': '#999999',
#         'weight': 0.5,
#         'opacity': 0.7
#     }
# ).add_to(layer1)

In [10]:
# 创建颜色映射
min_rider = bus_stops['Stop_Rider'].min()
max_rider = bus_stops['Stop_Rider'].max()
colormap = cm.LinearColormap(
    colors=['#e00a03','#fff782'],  # viridis颜色渐变
    vmin=min_rider,
    vmax=max_rider
)
colormap.caption = 'Bus Stop Ridership'
colormap.add_to(mm)



In [11]:
# 创建第一个图层（Layer 2）
layer1 = folium.FeatureGroup(name='Layer 1 - Points')

# 添加公交站点到地图
for _, stop in bus_stops.iterrows():
    rider = stop['Stop_Rider']
    size = 1 + 6 * (rider - min_rider) / (max_rider - min_rider)  # 半径范围5-20像素
    
    folium.CircleMarker(
        location=[stop['Y'], stop['X']],  # 注意坐标顺序：纬度在前
        radius=size,
        color=colormap(rider),
        fill=True,
        fill_color=colormap(rider),
        fill_opacity=0.7,
        opacity=0,
        weight=0
    ).add_to(layer1)



In [12]:
# 将图层添加到地图
#layer1.add_to(m)
layer1.add_to(mm)

# # 添加图层控制（允许用户切换图层）
# folium.LayerControl().add_to(m)
# 保存地图
mm.save('baltimore_bus_mapmmm.html')